In [2]:
import numpy as np
import pandas as pd
import scipy.io as sio
import os

In [9]:
# In this notebook we will try to detect anomalies in the ball bearing data set from https://csegroups.case.edu/bearingdatacente. Te description of the data set as mentiond in the website is as follows
"""
This website provides access to ball bearing test data for normal and faulty bearings.  Experiments were conducted using a 2 hp Reliance Electric motor, and acceleration data was measured at locations near to and remote from the motor bearings
"""

# !wget --no-check-certificate  https://csegroups.case.edu/bearingdatacenter/pages/normal-baseline-data/files/datafiles/97.mat
!wget --no-check-certificate  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
!wget --no-check-certificate  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
!wget --no-check-certificate  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/99.mat
!wget --no-check-certificate  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/100.mat

--2020-07-07 17:10:52--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/97.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 3903344 (3.7M)
Saving to: ‘97.mat.3’

100%[======================================>] 3,903,344   11.4MB/s   in 0.3s   

2020-07-07 17:10:52 (11.4 MB/s) - ‘97.mat.3’ saved [3903344/3903344]

--2020-07-07 17:10:54--  https://csegroups.case.edu/sites/default/files/bearingdatacenter/files/Datafiles/98.mat
Resolving csegroups.case.edu (csegroups.case.edu)... 129.22.25.84
Connecting to csegroups.case.edu (csegroups.case.edu)|129.22.25.84|:443... connected.
  Issued certificate has expired.
HTTP request sent, awaiting response... 200 OK
Length: 7742720 (7.4M)
Saving to: ‘98.mat’

100%[======================================>] 7,742,720   15.6MB/s 

In [10]:
!mkdir cwr_healthy
!mv *.mat cwr_healthy/

In [47]:
!ls -lahr cwr_healthy

total 34M
-rw-r----- 1 dsxuser dsxuser  15M Sep  7  2011 99.mat
-rw-r----- 1 dsxuser dsxuser 7.4M Sep  7  2011 98.mat
-rw-r----- 1 dsxuser dsxuser 3.8M Sep  7  2011 97.mat
-rw-r----- 1 dsxuser dsxuser 7.5M Sep  7  2011 100.mat
drwxr-x--- 3 dsxuser dsxuser 4.0K Jul  7 17:36 ..
drwxr-x--- 2 dsxuser dsxuser 4.0K Jul  7 17:11 .


In [48]:
# Lets check what output we can get if we read the file with scipy's loadmat method

mlf = sio.loadmat('./cwr_healthy/100.mat')
mlf

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Fri Jan 28 11:25:48 2000',
 '__version__': '1.0',
 '__globals__': [],
 'X100_DE_time': array([[ 0.01460308],
        [ 0.05444862],
        [ 0.10764554],
        ...,
        [-0.02357354],
        [ 0.00521538],
        [ 0.04777292]]),
 'X100_FE_time': array([[ 0.19292182],
        [ 0.16436364],
        [ 0.09081091],
        ...,
        [ 0.10930182],
        [ 0.05218545],
        [-0.00452   ]]),
 'X100RPM': array([[1725]], dtype=uint16)}

In [49]:
file = '100.mat'
file_id = file[:-4]
file_id

'100'

In [50]:
os.listdir('./cwr_healthy/')

['97.mat', '99.mat', '98.mat', '100.mat']

In [51]:
def read_folder(folder):
    data='dummy'
    skip = False
    for file in os.listdir(folder):
        file_id=file[:-4]
        mat_file_dict = sio.loadmat(folder+file)
        del data
        for key, value in mat_file_dict.items():
            if 'DE_time' in key or 'FE_time' in key:
                a = np.array(mat_file_dict[key])
                try:
                        data
                except NameError:
                    data = a
                else:
                    if (data.shape[0]!=a.shape[0]):
                        print ('skipping'  + file_id)
                        skip =True
                        continue
                    data = np.hstack((data,a))
        if skip:
            skip = False
            continue
        id = np.repeat(file_id, data.shape[0])
        id.shape = (id.shape[0],1)
        data = np.hstack((id, data))
            
        if data.shape[1]==2:
            zeros  = np.repeat(float(0), datashape[0])
            zeros.shape = (data.shape[0],1)
            data = np.hstack((data,  zeros))
        try: 
            result
        except NameError:
            result=data
        else:
            np.vstack((result,data))
        return result

In [52]:
result_healthy = read_folder('./cwr_healthy/') 
print(type(read_folder))

<class 'function'>


In [53]:
pdf= pd.DataFrame(result_healthy)

In [54]:
pdf.to_csv('result_healthy_pandas.csv', header=False, index= True)

In [55]:
!head result_healthy_pandas.csv

0,97,0.05319692307692307,0.14566727272727273
1,97,0.08866153846153846,0.09779636363636364
2,97,0.09971815384615385,0.05485636363636363
3,97,0.05862092307692307,0.03698181818181818
4,97,-0.004589538461538461,0.05444545454545455
5,97,-0.056951999999999996,0.02116181818181818
6,97,-0.0717636923076923,-0.003698181818181818
7,97,-0.05862092307692307,-0.010683636363636364
8,97,-0.04652123076923077,0.02938
9,97,-0.04985907692307692,0.10457636363636363
